Copyright © 2022-2023 HQS Quantum Simulations GmbH. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except
in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.

# Devices: GenericDevice, AllToAllDevice and SquareLatticeDevice

When working with quantum circuits it is often necessary to know the topology of a target quantum device. Device properties can also be used by backends, for example to accurately simulate a given quantum device.
qoqo/roqoqo defines an interface for obtaining the device topology. The interface is defined by roqoqo's `Device` trait. Additionally qoqo/roqoqo provides some simple devices that can be used to quickly define simple device topologies.



## GenericDevice
The `GenericDevice` is the most basic device. It simply contains all available gate operations, the corresponding gate times and the decoherence rate for each qubit in internal HashMaps. It can be used to create custom devices and as a device interchange format. As part of the `Device` interface, each device can be exported as a `GenericDevice` with the `to_generic_device` function.


In [1]:
from qoqo import devices
import numpy as np

# Create a two-qubit device
generic_device = devices.GenericDevice(2)
# Create a comparison two-qubit device with `RotateZ` and `CNOT` as the only gates and 1.0 as the default gate time
all_to_all = devices.AllToAllDevice(2, ["RotateZ"], ["CNOT"], 1.0)

generic_device.set_single_qubit_gate_time("RotateZ", 0, 1.0)
generic_device.set_single_qubit_gate_time("RotateZ", 1, 1.0)
generic_device.set_two_qubit_gate_time("CNOT", 0, 1, 1.0)
generic_device.set_two_qubit_gate_time("CNOT", 1, 0, 1.0)
# Set the decoherence rates directly
generic_device.set_qubit_decoherence_rates(0, np.array([[0.0, 0.0, 0.0],[0.0, 0.0, 0.0],[0.0, 0.0, 0.0]]))
generic_device.set_qubit_decoherence_rates(1, np.array([[0.0, 0.0, 0.0],[0.0, 0.0, 0.0],[0.0, 0.0, 0.0]]))
assert generic_device == all_to_all.generic_device()


## AllToAllDevice
The `AllToAllDevice` can be used to quickly create a device with all-to-all connectivity. It provides functions to set the gate time on all gates of a certain type and set the decoherence rates of all qubits. Contrary to the functions operating on single gates (`set_single_qubit_gate` etc.) those functions do not change the device but return a copy with these changes.

When setting attributes for *all* of the qubits on the device, the `AllToAllDevice` uses a builder pattern, in order for the user to be able to chain such calls. This is demonstrated below.


In [2]:
from qoqo import devices
import numpy as np

# Create a two-qubit device with `RotateZ` and `CNOT` as the only gates and 1.0 as the default gate time
all_to_all = devices.AllToAllDevice(2, ["RotateZ"], ["CNOT"], 1.0)

# Set a new time for all RotateZ gates and all CNOT gates
all_to_all = all_to_all.set_all_single_qubit_gate_times("RotateZ", 2.0).set_all_two_qubit_gate_times("CNOT", 0.1)

## SquareLatticeDevice
The `SquareLatticeDevice` can be used to quickly initialize a device with two-qubit operations available between next-neighbours on a square lattice. The same methods as `AllToAllDevice` are available.


In [3]:
from qoqo import devices

rows = 1
columns = 2

# Create a two-qubit device with `RotateZ` and `CNOT` as the only gates and 1.0 as the default gate time
square_lattice = devices.SquareLatticeDevice(rows, columns, ["RotateZ"], ["CNOT"], 1.0)


## Serialisation

The user can serialise and deserialise the devices using `to_json` and `from_json`.

In [4]:
from qoqo import devices

square_lattice = devices.SquareLatticeDevice(1, 2, ["RotateZ"], ["CNOT"], 1.0)
device_serialised = square_lattice.to_json()
device_deserialised = devices.SquareLatticeDevice.from_json(device_serialised)
assert square_lattice == device_deserialised

# Noise Models: ContinuousDecoherenceModel, DecoherenceOnGateModel, ImperfectReadout

When working with quantum computers it is often necessary to know the physical noise present of a target quantum device, particularly for NISQ devices. In roqoqo/qoqo, we have defined the three following noise models:


## ContinuousDecoherenceModel

The `ContinuousDecoherenceModel` is the noise model representing a continuous decoherence process on qubits. This noise model assumes that all qubits are constantly experiencing decoherence over time (e.g. due to coupling to the environment). The noise for each qubit can be different but only single qubit noise is included in the model.


In [5]:
from qoqo import noise_models
import numpy as np

continuous_model = noise_models.ContinuousDecoherenceModel()
continuous_model = continuous_model.add_damping_rate([0, 1, 2], 0.001)
continuous_model = continuous_model.add_dephasing_rate([0, 1, 2], 0.0005)
continuous_model = continuous_model.add_depolarising_rate([0, 1, 2], 0.0001)
continuous_model = continuous_model.add_excitation_rate([0, 1, 2], 0.0006)

# Access the underlying struqture operator
lindblad_noise = continuous_model.get_noise_operator()
lindblad_noise.add_operator_product(("0+", "0+"), 0.1)
new_continuous_model = noise_models.ContinuousDecoherenceModel(lindblad_noise)


## DecoherenceOnGateModel

The `DecoherenceOnGateModel` is the error model for noise that is only present on gate executions.Adds additional noise when specific gates (identified by hqslang name and qubits acted on) are executed. The noise is given in the form of a [struqture::spins::PlusMinusLindbladNoiseOperator] the same way it is for the ContinuousDecoherence model.

In [6]:
from qoqo import noise_models
from struqture_py.spins import PlusMinusLindbladNoiseOperator, PlusMinusProduct
import numpy as np

noise_model = noise_models.DecoherenceOnGateModel()
lindblad_noise = PlusMinusLindbladNoiseOperator()
lindblad_noise.add_operator_product(
   (PlusMinusProduct().z(0), PlusMinusProduct().z(0)),
   0.9)
lindblad_noise.add_operator_product(
   (PlusMinusProduct().z(1), PlusMinusProduct().z(1)),
   0.9)

noise_model = noise_model.set_two_qubit_gate_error(
    "CNOT", 0,1,
    lindblad_noise
)


Struqture python interface
Copyright © 2021-2023 HQS Quantum Simulations GmbH. All Rights Reserved.




## ImperfectReadoutModel

The `ImperfectReadoutModel` is the noise model representing readout errors. This noise model assumes that all qubits are constantly experiencing decoherence over time (e.g. due to coupling to the environment).

In [7]:
from qoqo import noise_models
import numpy as np

model = noise_models.ImperfectReadoutModel.new_with_uniform_error(3, 0.5, 0.5)
model = model.set_error_probabilites(2, 0.3, 0.7)
uniform_prob = model.prob_detect_0_as_1(0)
assert uniform_prob == 0.5
lower_prob = model.prob_detect_0_as_1(2)
assert lower_prob == 0.3
higher_prob = model.prob_detect_1_as_0(2)
assert higher_prob == 0.7


## Serialisation

The user can serialise and deserialise the devices using `to_json` and `from_json`.

In [8]:
from qoqo import noise_models

model = noise_models.ImperfectReadoutModel.new_with_uniform_error(3, 0.5, 0.5)
model = model.set_error_probabilites(2, 0.3, 0.7)
model_serialised = model.to_json()
model_deserialised = noise_models.ImperfectReadoutModel.from_json(model_serialised)
assert model == model_deserialised